In [1]:
!git clone https://github.com/dariogod/emotion-learning.git

Cloning into 'emotion-learning'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 151 (delta 49), reused 135 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (151/151), 586.45 KiB | 24.44 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
!pip install -r emotion-learning/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 253.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 248.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 260.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 257.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 236.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 243.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 195.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 197.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 71.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 174.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 143.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 223.6 MB/s eta 0:00:00


In [3]:
DATA_DIR="emotion-learning/data"
MODEL_NAME="answerdotai/ModernBERT-base"
EPOCHS=10
BATCH_SIZE=32
LEARNING_RATE=8e-5
SEED=42
EMOTION_LOSS_WEIGHT=0.7

# Create output directories
STANDARD_OUTPUT_DIR="./runs/standard_normalized"
EMOTION_OUTPUT_DIR="./runs/emotion_normalized"

In [4]:
!mkdir -p $STANDARD_OUTPUT_DIR
!mkdir -p $EMOTION_OUTPUT_DIR

In [5]:
!echo "Running standard training..."
!python emotion-learning/GLUE/emotion_finetune.py \
  --data_dir $DATA_DIR \
  --model_name $MODEL_NAME \
  --epochs $EPOCHS \
  --batch_size $BATCH_SIZE \
  --learning_rate $LEARNING_RATE \
  --output_dir $STANDARD_OUTPUT_DIR \
  --seed $SEED

Running standard training...
2025-04-28 12:34:34,059 - INFO - Random seed set to 42 for full reproducibility
2025-04-28 12:34:34,059 - INFO - PyTorch version: 2.5.1+cu124
2025-04-28 12:34:34,059 - INFO - CUDA available: True
2025-04-28 12:34:34,059 - INFO - CUDA version: 12.4
2025-04-28 12:34:34,059 - INFO - GPU device: NVIDIA GeForce RTX 4090
2025-04-28 12:34:34,059 - INFO - Running with arguments: Namespace(data_dir='emotion-learning/data', model_name='answerdotai/ModernBERT-base', epochs=10, learning_rate=8e-05, weight_decay=8e-06, batch_size=32, output_dir='./runs/standard_normalized', use_emotion=False, weighting_strategy='premise_bell_curve', curriculum=False, curriculum_key='premise_intensity', emotion_loss_weight=0.5, emotion_features=['premise_intensity', 'hypothesis_intensity', 'contrast'], logging_steps=100, seed=42, debug_loss=False)
2025-04-28 12:34:34,059 - INFO - Using emotion features: False
2025-04-28 12:34:34,059 - INFO - Loading tokenizer and model: answerdotai/Moder

In [6]:
import subprocess

weights = [round(x * 0.1, 1) for x in range(0, 11)]  # 0.0 to 1.0 in steps of 0.1

for weight in weights:
    output_dir = f"./notebook_sweep/emotion_weight_{weight}"
    cmd = [
        "python", "emotion-learning/GLUE/emotion_finetune.py",
        "--data_dir", DATA_DIR,
        "--model_name", MODEL_NAME,
        "--epochs", str(EPOCHS),
        "--batch_size", str(BATCH_SIZE),
        "--learning_rate", str(LEARNING_RATE),
        "--output_dir", output_dir,
        "--use_emotion",
        "--emotion_loss_weight", str(weight),
        "--emotion_features", "premise_intensity", "hypothesis_intensity", "contrast",
        "--debug_loss",
        "--seed", "42"
    ]
    print(f"Running for weight={weight}")
    subprocess.run(cmd)

Running for weight=0.0


2025-04-28 12:35:28,515 - INFO - Random seed set to 42 for full reproducibility
2025-04-28 12:35:28,515 - INFO - PyTorch version: 2.5.1+cu124
2025-04-28 12:35:28,515 - INFO - CUDA available: True
2025-04-28 12:35:28,515 - INFO - CUDA version: 12.4
2025-04-28 12:35:28,515 - INFO - GPU device: NVIDIA GeForce RTX 4090
2025-04-28 12:35:28,515 - INFO - Running with arguments: Namespace(data_dir='emotion-learning/data', model_name='answerdotai/ModernBERT-base', epochs=10, learning_rate=8e-05, weight_decay=8e-06, batch_size=32, output_dir='./notebook_sweep/emotion_weight_0.0', use_emotion=True, weighting_strategy='premise_bell_curve', curriculum=False, curriculum_key='premise_intensity', emotion_loss_weight=0.0, emotion_features=['premise_intensity', 'hypothesis_intensity', 'contrast'], logging_steps=100, seed=42, debug_loss=True)
2025-04-28 12:35:28,515 - INFO - Using emotion features: True
2025-04-28 12:35:28,515 - INFO - Loading tokenizer and model: answerdotai/ModernBERT-base
Some weights

{'eval_loss': 1.1590651273727417, 'eval_accuracy': 0.27, 'eval_f1_contradiction': 0.0547945205479452, 'eval_f1_entailment': 0.4262295081967213, 'eval_f1_neutral': 0.0, 'eval_f1_macro': 0.16034134291488883, 'eval_runtime': 0.4806, 'eval_samples_per_second': 416.14, 'eval_steps_per_second': 14.565, 'epoch': 1.0}



                                                
100%|██████████| 7/7 [00:00<00:00, 33.36it/s]2025-04-28 12:35:38,728 - INFO - Epoch 2.0: Metrics = {'eval_loss': 1.0544589757919312, 'eval_accuracy': 0.445, 'eval_f1_contradiction': 0.39316239316239315, 'eval_f1_entailment': 0.3888888888888889, 'eval_f1_neutral': 0.5142857142857142, 'eval_f1_macro': 0.4321123321123321, 'eval_runtime': 0.2628, 'eval_samples_per_second': 761.108, 'eval_steps_per_second': 26.639, 'epoch': 2.0}

                                             /opt/conda/lib/python3.11/site-packages/torch/nn/modules/linear.py:125: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or

{'eval_loss': 1.0544589757919312, 'eval_accuracy': 0.445, 'eval_f1_contradiction': 0.39316239316239315, 'eval_f1_entailment': 0.3888888888888889, 'eval_f1_neutral': 0.5142857142857142, 'eval_f1_macro': 0.4321123321123321, 'eval_runtime': 0.2628, 'eval_samples_per_second': 761.108, 'eval_steps_per_second': 26.639, 'epoch': 2.0}


...


                                                 
100%|██████████| 7/7 [00:00<00:00, 33.06it/s]2025-04-28 12:44:17,328 - INFO - Epoch 9.0: Metrics = {'eval_loss': 0.0, 'eval_accuracy': 0.24, 'eval_f1_contradiction': 0.14736842105263157, 'eval_f1_entailment': 0.35944700460829493, 'eval_f1_neutral': 0.045454545454545456, 'eval_f1_macro': 0.18408999037182397, 'eval_runtime': 0.2657, 'eval_samples_per_second': 752.614, 'eval_steps_per_second': 26.341, 'epoch': 9.0}

                                             /opt/conda/lib/python3.11/site-packages/torch/nn/modules/linear.py:125: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WOR

{'eval_loss': 0.0, 'eval_accuracy': 0.24, 'eval_f1_contradiction': 0.14736842105263157, 'eval_f1_entailment': 0.35944700460829493, 'eval_f1_neutral': 0.045454545454545456, 'eval_f1_macro': 0.18408999037182397, 'eval_runtime': 0.2657, 'eval_samples_per_second': 752.614, 'eval_steps_per_second': 26.341, 'epoch': 9.0}



                                                 
100%|██████████| 7/7 [00:00<00:00, 33.03it/s]2025-04-28 12:44:21,506 - INFO - Epoch 10.0: Metrics = {'eval_loss': 0.0, 'eval_accuracy': 0.24, 'eval_f1_contradiction': 0.14736842105263157, 'eval_f1_entailment': 0.35944700460829493, 'eval_f1_neutral': 0.045454545454545456, 'eval_f1_macro': 0.18408999037182397, 'eval_runtime': 0.2642, 'eval_samples_per_second': 757.088, 'eval_steps_per_second': 26.498, 'epoch': 10.0}

100%|██████████| 250/250 [00:43<00:00,  5.77it/s]
2025-04-28 12:44:22,652 - INFO - Saving final model to ./notebook_sweep/emotion_weight_1.0/final_model
2025-04-28 12:44:23,156 - INFO - Running final evaluation on test set
2025-04-28 12:44:23,156 - INFO - Random seed set to 42 for full reproducibility
2025-04-28 12:44:23,156 - INFO - Running final evaluation on test set...
2025-04-28 12:44:23,156 - INFO - Random seed set to 42 for full reproducibility
2025-04-28 12:44:23,164 - INFO - Test dataset loaded with 200 examples
202

{'eval_loss': 0.0, 'eval_accuracy': 0.24, 'eval_f1_contradiction': 0.14736842105263157, 'eval_f1_entailment': 0.35944700460829493, 'eval_f1_neutral': 0.045454545454545456, 'eval_f1_macro': 0.18408999037182397, 'eval_runtime': 0.2642, 'eval_samples_per_second': 757.088, 'eval_steps_per_second': 26.498, 'epoch': 10.0}
{'train_runtime': 43.3321, 'train_samples_per_second': 184.621, 'train_steps_per_second': 5.769, 'train_loss': 0.0, 'epoch': 10.0}


 57%|█████▋    | 4/7 [00:00<00:00, 30.52it/s]2025-04-28 12:44:23,455 - INFO - Loss normalization applied - Standard: 1.4102, Emotion: 0.2732, Ratio: 5.1629, Normalized emotion: 1.4102
2025-04-28 12:44:23,468 - INFO - Loss normalization applied - Standard: 1.5353, Emotion: 0.1900, Ratio: 8.0804, Normalized emotion: 1.5353
100%|██████████| 7/7 [00:00<00:00, 30.73it/s]
2025-04-28 12:44:23,470 - INFO - Final test metrics:
2025-04-28 12:44:23,470 - INFO -   accuracy: 0.2400
2025-04-28 12:44:23,470 - INFO -   f1_macro: 0.1841
2025-04-28 12:44:23,470 - INFO -   f1_contradiction: 0.1474
2025-04-28 12:44:23,470 - INFO -   f1_entailment: 0.3594
2025-04-28 12:44:23,470 - INFO -   f1_neutral: 0.0455
2025-04-28 12:44:23,470 - INFO - Random seed set to 42 for full reproducibility
2025-04-28 12:44:23,471 - INFO - Test examples by intensity: high=19, medium=80, low=101
2025-04-28 12:44:23,471 - INFO - Random seed set to 42 for full reproducibility
/opt/conda/lib/python3.11/site-packages/torch/nn/modul

In [9]:
import subprocess

cmd = [
    "python", "emotion-learning/GLUE/visualize.py",
    "--metrics_dir", "./notebook_sweep",
    "--output_dir", "./notebook_sweep/plots",
    "--compare_mode", "weight_sweep",
    "--smooth", "1"
]
subprocess.run(cmd)

Found metrics file: ./notebook_sweep/emotion_weight_0.0/emotion_weight_0.0_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.1/emotion_weight_0.1_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.2/emotion_weight_0.2_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.3/emotion_weight_0.3_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.4/emotion_weight_0.4_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.5/emotion_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.6/emotion_weight_0.6_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.7/emotion_weight_0.7_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.8/emotion_weight_0.8_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_0.9/emotion_weight_0.9_metrics.json
Found metrics file: ./notebook_sweep/emotion_weight_1.0/emotion_weight_1.0_metrics.json

Attempting to load 11 metrics files...
Suc

CompletedProcess(args=['python', 'emotion-learning/GLUE/visualize.py', '--metrics_dir', './notebook_sweep', '--output_dir', './notebook_sweep/plots', '--compare_mode', 'weight_sweep', '--smooth', '1'], returncode=0)